In [1]:
import numpy as np
import simpy as sp
import scipy.stats as stats

In [2]:
minutes2 = 2*24*60
minutes20 = 20*24*60
minutes22 = 22*24*60
def generate_customer():
    rand  = np.random.rand()
    if rand < 0.20: #NIA (Need immediate action)
        return "NIA"
    if rand < 1.00: #CA (Can Wait)
        return "CA"
def generate_NIA_time1():
    return(np.random.randint(3, 78))
def generate_NIA_time2():
    return(np.random.randint(5, 56))
def generate_CW_time1():
    return(np.random.randint(1, 30))
def generate_CW_time2():
    return(np.random.randint(2, 19))

In [3]:
class MonitoredPriorityResource(sp.PriorityResource):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.data = []
    def request(self, *args, **kwargs):
        if self.count != self.capacity:
            self.data.append((self._env.now, self.count + 1, len(self.queue)))
        else:
            self.data.append((self._env.now, self.count, len(self.queue) + 1))    
                    
        return super().request(*args, **kwargs)
    def release(self, *args, **kwargs):
        if len(self.queue) != 0:
            self.data.append((self._env.now, self.count, len(self.queue) - 1))
        else:
            self.data.append((self._env.now, self.count - 1, len(self.queue)))
            
        return super().release(*args, **kwargs)

In [4]:
avg1NIA = []
avg2NIA = []
avg1CW = []
avg2CW = []
dischargeCW = []
dischargeNIA = []
def patient(env, resource):
    global avg1NIA
    global avg2NIA
    global avg1CW
    global avg2CW
    global dischargeCW
    global dischargeNIA
    
    type_customer = generate_customer()
    if type_customer == "NIA":
        req = resource.request(priority=-3)
        arrive1 = env.now
        yield req
        enter1 = env.now
        avg1NIA.append(enter1 - arrive1)
            
        yield env.timeout(generate_NIA_time1())
        resource.release(req)
        
        req = resource.request(priority=-2)
        yield req  
        enter2 = env.now
        avg2NIA.append(enter2 - arrive1)

        yield env.timeout(generate_NIA_time2())
        exit2 = env.now
        dischargeNIA.append(exit2 - arrive1)
        
        resource.release(req)
        
    else: #CW
        req = resource.request(priority=-1)
        arrive1 = env.now
        yield req
        enter1 = env.now
        avg1CW.append(enter1 - arrive1)
            
        yield env.timeout(generate_CW_time1())        
        resource.release(req)
        
        req = resource.request(priority=-2)
        yield req  
        enter2 = env.now
        avg2CW.append(enter2 - arrive1)
            
        yield env.timeout(generate_CW_time2())
        exit2 = env.now
        dischargeCW.append(exit2 - arrive1)
            
        resource.release(req)
        
def warmup(env, resource):
    global avg1NIA
    global avg2NIA
    global avg1CW
    global avg2CW
    global dischargeCW
    global dischargeNIA
    
    yield env.timeout(minutes2)
    #clear statistics
    del resource.data[:len(resource.data)-1]
    resource.data[0] = (minutes2, resource.data[0][1], resource.data[0][2])
    avg1NIA.clear()
    avg2NIA.clear()
    avg1CW.clear()
    avg2CW.clear()
    dischargeCW.clear()
    dischargeNIA.clear()
        
def setup(env, resource):
    env.process(warmup(env, resource))
    while True:
        
        interarrival_time = np.random.randint(21, 60)
        yield env.timeout(interarrival_time)
        env.process(patient(env, resource))



In [5]:
no_of_doctors = 2

np.random.seed(344)       
env = sp.Environment()
doctors = MonitoredPriorityResource(env, capacity=no_of_doctors)
env.process(setup(env, doctors))
env.run(until = minutes22)

doctors.data.append((minutes22, len(doctors.users), len(doctors.queue)))

utilization = 0
for i in range(len(doctors.data) - 1):
    utilization += (doctors.data[i + 1][0] - doctors.data[i][0]) * doctors.data[i][1]
utilization /= (minutes20*no_of_doctors)

queue_len = 0
for i in range(len(doctors.data) - 1):
    queue_len += (doctors.data[i + 1][0] - doctors.data[i][0]) * doctors.data[i][2]
queue_len /= (minutes20)


print("--------------------------------------------------------")
print("Average time to first doctor for NIA patients = " + str('{:.4f}'.format(np.mean(avg1NIA))))
print("Average time to first doctor for CW patients = " + str('{:.4f}'.format(np.mean(avg1CW))))
print("Average time to second doctor for NIA patients = " + str('{:.4f}'.format(np.mean(avg2NIA))))
print("Average time to second doctor for CW patients = " + str('{:.4f}'.format(np.mean(avg2CW))))
print("Average time to discharge for NIA patients = " + str('{:.4f}'.format(np.mean(dischargeNIA))))
print("Average time to discharge for CW patients = " + str('{:.4f}'.format(np.mean(dischargeCW))))
print("--------------------------------------------------------")
print("Average doctor utilization = " + str('{:.4f}'.format(utilization)))
print("Average queue length = " + str('{:.4f}'.format(queue_len)))
print("--------------------------------------------------------")

--------------------------------------------------------
Average time to first doctor for NIA patients = 0.7973
Average time to first doctor for CW patients = 0.8014
Average time to second doctor for NIA patients = 43.2857
Average time to second doctor for CW patients = 15.8815
Average time to discharge for NIA patients = 74.4218
Average time to discharge for CW patients = 25.7805
--------------------------------------------------------
Average doctor utilization = 0.4362
Average queue length = 0.0222
--------------------------------------------------------


In [6]:
no_of_doctors = 1

np.random.seed(344)       
env = sp.Environment()
doctors = MonitoredPriorityResource(env, capacity=no_of_doctors)
env.process(setup(env, doctors))
env.run(until = minutes22)

doctors.data.append((minutes22, len(doctors.users), len(doctors.queue)))

utilization = 0
for i in range(len(doctors.data) - 1):
    utilization += (doctors.data[i + 1][0] - doctors.data[i][0]) * doctors.data[i][1]
utilization /= (minutes20*no_of_doctors)

queue_len = 0
for i in range(len(doctors.data) - 1):
    queue_len += (doctors.data[i + 1][0] - doctors.data[i][0]) * doctors.data[i][2]
queue_len /= (minutes20)


print("--------------------------------------------------------")
print("Average time to first doctor for NIA patients = " + str('{:.4f}'.format(np.mean(avg1NIA))))
print("Average time to first doctor for CW patients = " + str('{:.4f}'.format(np.mean(avg1CW))))
print("Average time to second doctor for NIA patients = " + str('{:.4f}'.format(np.mean(avg2NIA))))
print("Average time to second doctor for CW patients = " + str('{:.4f}'.format(np.mean(avg2CW))))
print("Average time to discharge for NIA patients = " + str('{:.4f}'.format(np.mean(dischargeNIA))))
print("Average time to discharge for CW patients = " + str('{:.4f}'.format(np.mean(dischargeCW))))
print("--------------------------------------------------------")
print("Average doctor utilization = " + str('{:.4f}'.format(utilization)))
print("Average queue length = " + str('{:.4f}'.format(queue_len)))
print("--------------------------------------------------------")

--------------------------------------------------------
Average time to first doctor for NIA patients = 11.4558
Average time to first doctor for CW patients = 101.5439
Average time to second doctor for NIA patients = 80.4490
Average time to second doctor for CW patients = 119.0771
Average time to discharge for NIA patients = 110.7211
Average time to discharge for CW patients = 128.7356
--------------------------------------------------------
Average doctor utilization = 0.8531
Average queue length = 2.2639
--------------------------------------------------------


In [7]:
no_of_doctors = 3

np.random.seed(344)       
env = sp.Environment()
doctors = MonitoredPriorityResource(env, capacity=no_of_doctors)
env.process(setup(env, doctors))
env.run(until = minutes22)

doctors.data.append((minutes22, len(doctors.users), len(doctors.queue)))

utilization = 0
for i in range(len(doctors.data) - 1):
    utilization += (doctors.data[i + 1][0] - doctors.data[i][0]) * doctors.data[i][1]
utilization /= (minutes20*no_of_doctors)

queue_len = 0
for i in range(len(doctors.data) - 1):
    queue_len += (doctors.data[i + 1][0] - doctors.data[i][0]) * doctors.data[i][2]
queue_len /= (minutes20)


print("--------------------------------------------------------")
print("Average time to first doctor for NIA patients = " + str('{:.4f}'.format(np.mean(avg1NIA))))
print("Average time to first doctor for CW patients = " + str('{:.4f}'.format(np.mean(avg1CW))))
print("Average time to second doctor for NIA patients = " + str('{:.4f}'.format(np.mean(avg2NIA))))
print("Average time to second doctor for CW patients = " + str('{:.4f}'.format(np.mean(avg2CW))))
print("Average time to discharge for NIA patients = " + str('{:.4f}'.format(np.mean(dischargeNIA))))
print("Average time to discharge for CW patients = " + str('{:.4f}'.format(np.mean(dischargeCW))))
print("--------------------------------------------------------")
print("Average doctor utilization = " + str('{:.4f}'.format(utilization)))
print("Average queue length = " + str('{:.4f}'.format(queue_len)))
print("--------------------------------------------------------")

--------------------------------------------------------
Average time to first doctor for NIA patients = 0.0000
Average time to first doctor for CW patients = 0.0087
Average time to second doctor for NIA patients = 42.6510
Average time to second doctor for CW patients = 15.0070
Average time to discharge for NIA patients = 74.8446
Average time to discharge for CW patients = 24.9200
--------------------------------------------------------
Average doctor utilization = 0.2946
Average queue length = 0.0002
--------------------------------------------------------
